In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import json
from pyzipcode import ZipCodeDatabase
import string
import colour

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import StratifiedKFold

from sklearn.externals import joblib


from datetime import datetime

### some defs

In [3]:
trash=[1,2]
[t1,t2]=trash
print t1

1


In [4]:
#only use those areas where I have pricing information.

#normalize percentage increase by area.
def samNormalize(I_array):
    normval=np.nanmean(I_array)
    return np.divide(I_array,normval)

def samLatLon(I_array, weights=1):
    #determine mean
    citycenter= np.nanmean(I_array,axis=0)
    
    #determine distance from mean
    O_array=I_array
    O_array[:,0]=np.subtract(I_array[:,0],citycenter[0])
    O_array[:,1]=np.subtract(I_array[:,1],citycenter[1])
    dists=np.sqrt(np.add(np.square(O_array[:,0]),np.square(O_array[:,1])))
    return samNormalize(dists)

def samRank(I_array):
    #from High to Low
    order = I_array.argsort()[::-1]
    return order.argsort()

In [13]:
def samlinearFit(df, stopdate, predictdate):
    #create linear fit for each row
    full_price_df=df.iloc[:,6:71]
    
    normdate=np.datetime64('2014-01')
    
    #determine dates
    dates=[np.datetime64(date) for date in list(full_price_df.columns.values)]
    id_pr=np.argmax(dates>predictdate)
    id_in=np.argmax(dates>stopdate)
    id_no=np.argmax(dates>normdate)
    
    #build dataframe, normalize by value at input time
    full_price_df = full_price_df.div(full_price_df.iloc[:,id_no],axis='index')
    myrange,=np.where(dates<stopdate)
    newdf=full_price_df.iloc[:,myrange]
    
    #establish variables
    flag=[]
    
    slope=np.empty(df.shape)
    inter=np.empty(df.shape)
    NANs=np.empty([1,2])
    NANs[:]=np.NAN
    print NANs
    for i_, row in newdf.iterrows():
        if i_%500==0:
            print i_
        
        for j_, cutoff in enumerate(myrange):
            i_df=pd.DataFrame({'price':row.values[:j_],'date':range(myrange.size)[:j_]})
            
            try:
                result = sm.ols(formula="price ~ date", data=i_df).fit()
                [inter[i_,j_], slope[i_, j_]]=result.params.values
                    
            except:
                [inter[i_,j_], slope[i_, j_]]=NANs[0]
                flag.append(i_)
        
    
    return inter, slope, newdf, flag

In [70]:
print df.shape[0]



10728


### Load data

In [14]:
#get real estate data
# http://www.zillow.com/research/data/
filename='../../Data/Zillow/Zip/Zip_MedianRentalPricePerSqft_AllHomes.csv'
filename='../../Data/Zillow/Zip/Zip_MedianListingPricePerSqft_AllHomes.csv'
with open(filename,'r') as zf:
    df=pd.read_csv(zf,header=0)

In [15]:
#separate into pre/post 2014
#Use info from pre-2014 to train model based on 2015 real estate estimates.
featurenames=['Intercept','Slope']
inter14, slope14, new_df14, flag14=samlinearFit(df,np.datetime64('2014-01'),np.datetime64('2016-01'))

[[ nan  nan]]
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500


In [16]:
#separate into pre/post 2014
#Use info from pre-2014 to train model based on 2015 real estate estimates.
featurenames=['Intercept','Slope']
inter16, slope16, new_df16, flag16=samlinearFit(df,np.datetime64('2016-06'),np.datetime64('2016-01'))

[[ nan  nan]]
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500


In [17]:
#get real estate data
# http://www.zillow.com/research/data/
filename='../../Data/Zillow/Zip/Zip_MedianRentalPricePerSqft_AllHomes.csv'
with open(filename,'r') as zf:
    df=pd.read_csv(zf,header=0)

In [18]:
#separate into pre/post 2014
#Use info from pre-2014 to train model based on 2015 real estate estimates.
featurenames=['Intercept','Slope']
inter_rent14, slope_rent14, new_df_rent14, flag14=samlinearFit(df,np.datetime64('2014-01'),np.datetime64('2016-01'))

[[ nan  nan]]
0
500
1000
1500


In [19]:
#separate into pre/post 2014
#Use info from pre-2014 to train model based on 2015 real estate estimates.
featurenames=['Intercept','Slope']
inter_rent16, slope_rent16, new_df_rent16, flag16=samlinearFit(df,np.datetime64('2016-06'),np.datetime64('2016-01'))

[[ nan  nan]]
0
500
1000
1500


### Start creating features.

In [89]:

zips=df['RegionName'].values
ranks=np.ones((zips.size))*np.NAN
zcdb = ZipCodeDatabase()

lat=[]
lon=[]
state=[]
for z in zips:
    try:
        zc=zcdb[z]
    except:
        lat.append(np.NAN)
        lon.append(np.NAN)
        state.append(np.NAN)
        continue
    state.append(state_nums[str(zc.state)])
    lat.append(zc.latitude)
    lon.append(zc.longitude)

state=np.asarray(state)
lat=np.asarray(lat)
lon=np.asarray(lon)
normPC=ranks

percChange=percChange.values
for i in range(50):
    index=np.where(state==i)
    
    normPC[index]=samNormalize(percChange[index])

NameError: name 'percChange' is not defined

In [62]:
slopeInt, flag, perc, trash= samlinearFit(df,np.datetime64('2016-06'),np.datetime64('2016-01'))

0
2000
4000
6000
8000
10000


In [66]:
plt.close()


print state
# r_io=np.column_stack((slopeInt,dists,percChange,rank2))
r_io=np.column_stack((slopeInt,state,lon,lat,normPC,ranks))
r_io=r_io[~np.isnan(r_io).any(axis=1)]

fig, axes=plt.subplots(ncols=1, nrows=2, figsize=[6,5])
axes[0].scatter(normPC, slopeInt[:,0])
# axes[0].set_ylim([-.02, .02])
axes[1].hist(r_io[:,-2],100)

plt.show()
stop

ip=r_io[:,0:5]
print ip.shape
op=r_io[:,-1]
skf = StratifiedKFold(op, n_folds=3)
for fold, (tr, te) in enumerate(skf):
    rnf = RandomForestClassifier(n_estimators=100, max_depth=None,
        min_samples_split=50, max_features=2, random_state=1, max_leaf_nodes=30)
    rnf2=rnf
    
    rnf.fit(ip[tr,:],op[tr])
#     print rnf.predict(r_io[0:100,0:3])
    newte=te[op[te]==1]
#     te=newte
    print ('Fold:%i, G0:%.2f, G1:%.2f, G2:%.2f, G3:%.2f, G4:%.2f, RNF: %.3f' %
        (fold, accuracy_score(op[te], rnf.predict(ip[te,:])*0.+0.), 
         accuracy_score(op[te], rnf.predict(ip[te,:])*0.+1.), 
         accuracy_score(op[te], rnf.predict(ip[te,:])*0.+2.), 
         accuracy_score(op[te], rnf.predict(ip[te,:])*0.+3.), 
         accuracy_score(op[te], rnf.predict(ip[te,:])*0.+4.), 
         accuracy_score(op[te], rnf.predict(ip[te,:]))))


plot_learning_curve(rnf2,'Random Forest',ip,op,(0,1.0))
plt.show()

[ 12.  42.  12. ...,  17.  31.  16.]


NameError: name 'normPC' is not defined

In [5]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
state_abrevs=dict((v, k) for k, v in states.iteritems())
state_nums=dict((k[0],id_) for id_, k  in enumerate(states.iteritems()))